In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

In [5]:
# Send HTTP GET request to Wikipedia snapshot page
response = requests.get(static_url, headers=headers)

# Print status code (200 means success)
print("Status Code:", response.status_code)

# Preview the first 500 characters of the HTML page
print(response.text[:500])


Status Code: 200
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


In [6]:
# Create a BeautifulSoup object from the response text
soup = BeautifulSoup(response.text, "html.parser")

# Print the page title to check if it's working
print(soup.title)
print(soup.title.string)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [7]:
# Find all <table> elements on the page
html_tables = soup.find_all('table')
print("Number of tables:", len(html_tables))


Number of tables: 25


In [8]:
# Select the 3rd table (Python index 2)
first_launch_table = html_tables[2]

# Print out just the first 500 characters to check
print(str(first_launch_table)[:500])


<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[<


In [9]:
table_head = first_launch_table.find_all('th')
column_names = [extract_column_from_header(i) for i in table_head]
print(column_names)


['Flight No.', 'Date and time ( )', '', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', '', None, None, None, None, None, None, None]


In [10]:
column_names = []

# Find all header cells in the launch table
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:   # skip empty ones
        column_names.append(name)

# Print the result
print(column_names)


['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [11]:
launch_dict = dict.fromkeys(column_names)

# Remove unwanted column
del launch_dict['Date and time ( )']

# Initialize each key with an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Add extra keys
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []


In [18]:
extracted_row = 0

# Loop through all Falcon 9/Heavy tables on the page
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    
    # For each row in the table
    for rows in table.find_all("tr"):
        
        # Check if the row has a launch number in <th>
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()  # only process if it's a number
        else:
            flag = False
        
        # Get all table cells
        row = rows.find_all('td')
        
        if flag:
            extracted_row += 1
            
            # --- Extract values ---
            datetimelist = date_time(row[0])
            date = datetimelist[0].strip(',')
            time = datetimelist[1]
            
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string if row[1].a else row[1].get_text(strip=True)
            
            launch_site = row[2].a.string if row[2].a else row[2].get_text(strip=True)
            payload = row[3].a.string if row[3].a else row[3].get_text(strip=True)
            payload_mass = get_mass(row[4])
            orbit = row[5].a.string if row[5].a else row[5].get_text(strip=True)
            customer = row[6].a.string if row[6].a else row[6].get_text(strip=True)
            launch_outcome = list(row[7].strings)[0]
            booster_landing = landing_status(row[8])
            
            # --- Append values into dictionary ---
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            launch_dict['Version Booster'].append(bv)
            launch_dict['Launch site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Booster landing'].append(booster_landing)

print("Total rows extracted:", extracted_row)


Total rows extracted: 121


In [19]:
df = pd.DataFrame(launch_dict)
print(df.head())          # first 5 rows
print(df.shape)           # number of rows and columns

  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1          2       CCAFS                                Dragon            0   
2          3       CCAFS                                Dragon       525 kg   
3          4       CCAFS                          SpaceX CRS-1     4,700 kg   
4          5       CCAFS                          SpaceX CRS-2     4,877 kg   

  Orbit Customer Launch outcome   Version Booster Booster landing  \
0   LEO   SpaceX      Success\n  F9 v1.07B0003.18         Failure   
1   LEO     NASA        Success  F9 v1.07B0004.18         Failure   
2   LEO     NASA        Success  F9 v1.07B0005.18    No attempt\n   
3   LEO     NASA      Success\n  F9 v1.07B0006.18      No attempt   
4   LEO     NASA      Success\n  F9 v1.07B0007.18    No attempt\n   

              Date   Time  
0      4 June 2010  18:45  
1  8 December 2010  15:43  
2      22 May 2012  07:44 

In [20]:
df.to_csv('spacex_web_scraped.csv', index=False)